In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import pandas as pd

# Strategy

In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import pandas as pd
# Import the backtrader platform
import backtrader as bt
# btanalyzers


# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
#         bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
#         bt.indicators.WeightedMovingAverage(self.datas[0], period=25).subplot = True
#         bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
#         bt.indicators.ATR(self.datas[0]).plot = False
        
        self.sma = bt.indicators.MovingAverageSimple(self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return 
        
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None
    
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        if not self.position:
            if self.dataclose[0] > self.sma[0]:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                    
        else:
            # Already in the market ... we might sell
            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
                
    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
         (self.params.maperiod, self.broker.getvalue()), doprint=True)

In [ ]:

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere

datapath = os.path.join('../data/raw/demo/orcl-1995-2014.txt')

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

# Running  

In [ ]:

        
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
# Add a strategy
strats = cerebro.optstrategy(
    TestStrategy,
    maperiod=range(10, 31))

data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-01-01'])
cerebro.adddata(data)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.broker.setcommission(commission=0.00)

cerebro.broker.setcash(100000.0)

cerebro.broker.set_slippage_fixed(0.00, slip_open=True, slip_limit=True, slip_match=True, slip_out=True)


# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run(maxcpus=1)

for ret in results:
    for r in ret:
        print('MA Period %.2f Sharpe Ratio: %.7f '% 
              (r.params.maperiod,
              r.analyzers.mysharpe.get_analysis().get('sharperatio'))
             )
        
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# In sample

In [ ]:

# Create a cerebro entity
cerebro = bt.Cerebro()

cerebro.addstrategy(TestStrategy)

data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-01-01'])
cerebro.adddata(data)


cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.broker.setcommission(commission=0.001)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.broker.setcash(100000.0)
cerebro.broker.set_slippage_fixed(0.00, slip_open=True, slip_limit=True, slip_match=True, slip_out=True)


# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run(maxcpus=1)

for r in results:
    print('MA Period %.2f Sharpe Ratio: %.7f '% 
          (r.params.maperiod,
          r.analyzers.mysharpe.get_analysis().get('sharperatio'))
         )
        
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.plot()

# Out of Sample 

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()



data = bt.feeds.PandasData(dataname = df['2018-01-01':'2018-06-01'])

# Add a strategy
cerebro.addstrategy(TestStrategy)
cerebro.adddata(data)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.broker.setcommission(commission=0.001)
cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, _name='mysharpe')
cerebro.broker.setcash(100000.0)
cerebro.broker.set_slippage_fixed(0.00, slip_open=True, slip_limit=True, slip_match=True, slip_out=True)


# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run(maxcpus=1)

for r in results:
    print('MA Period {} Sharpe Ratio: {} '.format(r.params.maperiod,
          r.analyzers.mysharpe.get_analysis().get('sharperatio'))
         )
        
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.plot()